In [21]:
import pandas as pd

sheet_id = "1DERvsSuI838s8ZRNG69sP4Pz2tgnHvoTvGljUk0ldQc"  # your Google Sheet ID
People = "0"
Marriages = "1981493742"  # the gid of the 'Marriages' sheet

url = "https://docs.google.com/spreadsheets/d/{}/export?format=csv&gid={}"

marriages_df = pd.read_csv(url.format(sheet_id,Marriages))
people_df = pd.read_csv(url.format(sheet_id,People))


In [ ]:
import pandas as pd
import plotly.graph_objs as go
import dash
from dash import dcc, html
import dash_cytoscape as cyto

# Create a map of person names to their IDs and generation
name_to_id = {f"{row['First']} {row['Last']}": row['P_id'] for _, row in people_df.iterrows()}

# Create nodes for each person
nodes = []
edges = []

# Create nodes for people (name as label, id as unique identifier, xpos from the csv)
for _, row in people_df.iterrows():
    nodes.append({
        'data': {'id': row['P_id'], 'label': f"{row['First']} {row['Last']}"},
        'position': {'x': row['xpos']*100, 'y': row['Generation'] * 100}  # Adjust y based on generation
    })

# Create marriage edges and child edges
for _, row in marriages_df.iterrows():
    person_a_id = name_to_id[row['PersonA']]
    person_b_id = name_to_id[row['PersonB']]
    
    # Create edges for the marriage between PersonA and PersonB
    edges.append({
        'data': {'source': person_a_id, 'target': person_b_id, 'label': 'Marriage'},
        'style': {'line-color': 'blue'}
    })
    
    # Create child edges from the middle of the marriage line
    # Average x position of the two parents to place the child edge
    marriage_x = (people_df.loc[people_df['P_id'] == person_a_id, 'xpos'].values[0] + 
                  people_df.loc[people_df['P_id'] == person_b_id, 'xpos'].values[0]) / 2
    
    for child in row['Children'].split(";"):
        child_name = child.strip()
        child_id = name_to_id[child_name] if child_name in name_to_id else None
        if child_id:
            edges.append({
                'data': {'source': person_a_id, 'target': child_id, 'label': 'Child'},
                'style': {'line-color': 'green'},
                'position': {'x': marriage_x, 'y': (row['Generation'] + 1) * 100}  # Position child
            })

# Create Dash App
app = dash.Dash(__name__)

# Cytoscape layout options
layout = {
    'name': 'preset',  # Use preset positions
    'animate': True
}

# Dash Cytoscape component
app.layout = html.Div([
    cyto.Cytoscape(
        id='family-tree',
        elements=nodes + edges,
        layout=layout,
        style={'width': '100%', 'height': '600px'}
    )
])

# Run the app
if __name__ == '__main__':
    app.run(debug=True)


500.0
2500.0
500.0


In [45]:
import pandas as pd
import dash
from dash import dcc, html
import dash_cytoscape as cyto



# Create a map of person names to their IDs and generation
name_to_id = {f"{row['First']} {row['Last']}": row['P_id'] for _, row in people_df.iterrows()}

# Create nodes for each person
nodes = []
edges = []

# Create nodes for people (name as label, id as unique identifier, xpos from the csv)
for _, row in people_df.iterrows():
    nodes.append({
        'data': {'id': row['P_id'], 'label': f"{row['First']} {row['Last']}"},
        'position': {'x': row['xpos'], 'y': row['Generation'] * 100}  # Adjust y based on generation
    })

# Create marriage edges and child edges
for _, row in marriages_df.iterrows():
    person_a_id = name_to_id[row['PersonA']]
    person_b_id = name_to_id[row['PersonB']]
    
    # Create edges for the marriage between PersonA and PersonB
    edges.append({
        'data': {'source': person_a_id, 'target': person_b_id, 'label': 'Marriage'},
        'style': {'line-color': 'blue'}
    })
    
    # Calculate the middle of the marriage line (average x-position of both parents)
    marriage_x = (people_df.loc[people_df['P_id'] == person_a_id, 'xpos'].values[0] + 
                  people_df.loc[people_df['P_id'] == person_b_id, 'xpos'].values[0]) / 2
    
    # Set y-position for children (next generation)
    children_generation_y = (row['Generation'] + 1) * 100
    
    # Create a small marriage node at the center
    marriage_node_id = f"marriage_{row['M_id']}"
    nodes.append({
        'data': {'id': marriage_node_id, 'label': f"Marriage: {row['PersonA']} & {row['PersonB']}"},
        'position': {'x': marriage_x, 'y': children_generation_y - 50},  # Position it slightly above children
        'style': {'width': 10, 'height': 10, 'background-color': 'blue', 'shape': 'ellipse'}
    })
    
    # Create child edges from the middle of the marriage line
    for child in row['Children'].split(","):
        child_name = child.strip()
        child_id = name_to_id[child_name] if child_name in name_to_id else None
        if child_id:
            edges.append({
                'data': {'source': marriage_node_id, 'target': child_id, 'label': 'Child'},
                'style': {'line-color': 'green'}
            })
            
            # Create nodes for children and position them
            nodes.append({
                'data': {'id': child_id, 'label': f"{child_name}"},
                'position': {'x': marriage_x, 'y': children_generation_y}  # Position child at middle of marriage line
            })

# Create Dash App
app = dash.Dash(__name__)

# Cytoscape layout options
layout = {
    'name': 'preset',  # Use preset positions
    'animate': True
}

# Dash Cytoscape component
app.layout = html.Div([
    cyto.Cytoscape(
        id='family-tree',
        elements=nodes + edges,
        layout=layout,
        style={'width': '100%', 'height': '600px'},
        # Hover events to show details for marriage nodes
        
    ),
    html.P(id='cytoscape-tapNodeData-output'),
    html.P(id='cytoscape-tapEdgeData-output'),
    html.P(id='cytoscape-mouseoverNodeData-output'),
    html.P(id='cytoscape-mouseoverEdgeData-output')
])



@callback(Output('cytoscape-tapNodeData-output', 'children'),
              Input('cytoscape-event-callbacks-2', 'tapNodeData'))
def displayTapNodeData(data):
    if data:
        return "You recently clicked/tapped the city: " + data['label']


@callback(Output('cytoscape-tapEdgeData-output', 'children'),
              Input('cytoscape-event-callbacks-2', 'tapEdgeData'))
def displayTapEdgeData(data):
    if data:
        return "You recently clicked/tapped the edge between " + \
               data['source'].upper() + " and " + data['target'].upper()


@callback(Output('cytoscape-mouseoverNodeData-output', 'children'),
              Input('cytoscape-event-callbacks-2', 'mouseoverNodeData'))
def displayTapNodeData(data):
    if data:
        return "You recently hovered over the city: " + data['label']


@callback(Output('cytoscape-mouseoverEdgeData-output', 'children'),
              Input('cytoscape-event-callbacks-2', 'mouseoverEdgeData'))
def displayTapEdgeData(data):
    if data:
        return "You recently hovered over the edge between " + \
               data['source'].upper() + " and " + data['target'].upper()


# Run the app
if __name__ == '__main__':
    app.run(debug=True)


In [46]:
from dash import Dash, html, Input, Output, callback
import dash_cytoscape as cyto

app = Dash()

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}


nodes = [
    {
        'data': {'id': short, 'label': label},
        'position': {'x': 20*lat, 'y': -20*long}
    }
    for short, label, long, lat in (
        ('la', 'Los Angeles', 34.03, -118.25),
        ('nyc', 'New York', 40.71, -74),
        ('to', 'Toronto', 43.65, -79.38),
        ('mtl', 'Montreal', 45.50, -73.57),
        ('van', 'Vancouver', 49.28, -123.12),
        ('chi', 'Chicago', 41.88, -87.63),
        ('bos', 'Boston', 42.36, -71.06),
        ('hou', 'Houston', 29.76, -95.37)
    )
]

edges = [
    {'data': {'source': source, 'target': target}}
    for source, target in (
        ('van', 'la'),
        ('la', 'chi'),
        ('hou', 'chi'),
        ('to', 'mtl'),
        ('mtl', 'bos'),
        ('nyc', 'bos'),
        ('to', 'hou'),
        ('to', 'nyc'),
        ('la', 'nyc'),
        ('nyc', 'bos')
    )
]


default_stylesheet = [
    {
        'selector': 'node',
        'style': {
            'background-color': '#BFD7B5',
            'label': 'data(label)'
        }
    }
]


app.layout = html.Div([
    cyto.Cytoscape(
        id='cytoscape-event-callbacks-2',
        layout={'name': 'preset'},
        elements=edges+nodes,
        stylesheet=default_stylesheet,
        style={'width': '100%', 'height': '450px'}
    ),
    html.P(id='cytoscape-tapNodeData-output'),
    html.P(id='cytoscape-tapEdgeData-output'),
    html.P(id='cytoscape-mouseoverNodeData-output'),
    html.P(id='cytoscape-mouseoverEdgeData-output')
])


@callback(Output('cytoscape-tapNodeData-output', 'children'),
              Input('cytoscape-event-callbacks-2', 'tapNodeData'))
def displayTapNodeData(data):
    if data:
        return "You recently clicked/tapped the city: " + data['label']


@callback(Output('cytoscape-tapEdgeData-output', 'children'),
              Input('cytoscape-event-callbacks-2', 'tapEdgeData'))
def displayTapEdgeData(data):
    if data:
        return "You recently clicked/tapped the edge between " + \
               data['source'].upper() + " and " + data['target'].upper()


@callback(Output('cytoscape-mouseoverNodeData-output', 'children'),
              Input('cytoscape-event-callbacks-2', 'mouseoverNodeData'))
def displayTapNodeData(data):
    if data:
        return "You recently hovered over the city: " + data['label']


@callback(Output('cytoscape-mouseoverEdgeData-output', 'children'),
              Input('cytoscape-event-callbacks-2', 'mouseoverEdgeData'))
def displayTapEdgeData(data):
    if data:
        return "You recently hovered over the edge between " + \
               data['source'].upper() + " and " + data['target'].upper()


if __name__ == '__main__':
    app.run(debug=True)


In [3]:
import networkx as nx

G = nx.DiGraph()

# Add nodes and edges (marriages and children)
G.add_node("P1")
G.add_node("P2")
G.add_edge("P1", "M1")  # P1 part of marriage
G.add_edge("P2", "M1")
G.add_edge("M1", "C1")  # Marriage has child

# Use Graphviz to compute layout
pos = nx.nx_agraph.graphviz_layout(G, prog='dot')  # or use 'neato'/'twopi'

# Use pos to set node positions in Cytoscape
pos


{'P1': (27.0, 162.0),
 'P2': (99.0, 162.0),
 'M1': (63.0, 90.0),
 'C1': (63.0, 18.0)}